In [ ]:
import numpy as np
import cv2
import sys
from numba import jit
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
@jit
def _thinningIteration(im, iter_):
    M = np.zeros(im.shape, np.uint8)
    h, w = im.shape
    for i in range(1, h - 1):
        for j in range(1, w - 1):
            p2 = im[i - 1, j]
            p3 = im[i - 1, j + 1]
            p4 = im[i, j + 1]
            p5 = im[i + 1, j + 1]
            p6 = im[i + 1, j]
            p7 = im[i + 1, j - 1]
            p8 = im[i, j - 1]
            p9 = im[i - 1, j - 1]
            A = (p2 == 0 and p3 == 1) + (p3 == 0 and p4 == 1) + \
                (p4 == 0 and p5 == 1) + (p5 == 0 and p6 == 1) + \
                (p6 == 0 and p7 == 1) + (p7 == 0 and p8 == 1) + \
                (p8 == 0 and p9 == 1) + (p9 == 0 and p2 == 1)
            B = p2 + p3 + p4 + p5 + p6 + p7 + p8 + p9
            m1 = (p2 * p4 * p6) if (iter_ == 0) else (p2 * p4 * p8)
            m2 = (p4 * p6 * p8) if (iter_ == 0) else (p2 * p6 * p8)
            if A == 1 and B >= 2 and B <=6 and m1 == 0 and m2 == 0:
                M[i, j] = 1    
    M = ~M
    return np.bitwise_and(np.uint8(im), np.uint8(M))

In [ ]:
@jit
def thinning(src):
	dst = src.copy()
	prev = np.zeros(src.shape[:2], np.uint8)
	diff = None

	while True:
		dst = _thinningIteration(dst, 0)
		dst = _thinningIteration(dst, 1)
		diff = np.absolute(dst - prev)
		prev = dst.copy()
		if np.sum(diff) == 0:
			break

	return dst * 255

In [ ]:
if __name__ == "__main__":
        src = cv2.imread("Desktop\Sketch Fill\sketchapp\hp.bmp")
        if src.all() == None:
            sys.exit()
        bw = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
        _, bw2 = cv2.threshold(bw, 127, 255, cv2.THRESH_BINARY)
        #bw2 = bw.copy() // 255
        bw2 = thinning(bw2)
        bw2 = bw2*255
        cv2.imshow("src", bw)
        cv2.imshow("thinning", bw2)
        cv2.imwrite('thinning.jpg', bw2)
        cv2.waitKey()
        '''plt.imshow(bw2, cmap=plt.cm.gray)
        plt.title('Thinning')
        plt.axis('off')
        plt.savefig('op1.jpg')
        plt.show()'''

In [ ]:
main = bw2
row, col= main.shape

In [ ]:
import copy
#main = laplacian
row, col= main.shape
d = copy.deepcopy(main)
for c in range(col):
    r = 0
    while r<row:
        s = r
        while d[r][c]==0:
            d[r][c] = 255
            r += 1
        if s<r:
            d[s+(r-s)//2][c]=0
        r += 1
#plt.imshow(d, "gray")

e = copy.deepcopy(main)
for r in range(row):
    c = 0
    while c<col:
        s = c
        while e[r][c]==0:
            e[r][c] = 255
            c += 1
        if s<c:
            e[r][s+(c-s)//2]=0
        c += 1

# import scipy.misc
# scipy.misc.imsave('foo.jpg', d*e)
for i in range(row):
    for j in range(col):
        main[i][j] = 0 if d[i][j]==0 or e[i][j]==0 else 255
#main = d
plt.imshow(main, "gray")
plt.axis("off")
plt.savefig('3.jpg')

In [ ]:
#cols
main = np.insert(main, [0,row], 255, axis=1)
main.shape

In [ ]:
#rows
main = np.insert(main, [0, row], [np.array([255 for i in range(col+2)])], axis=0)
main.shape

In [ ]:
def find_8_neighbors(main, pos_x, pos_y):
    count = 0
    for i in range(pos_x-1, pos_x+2):
        for j in range(pos_y-1, pos_y+2):
            if main[i][j] < 255:
                count += 1                
    return count

In [ ]:
def find_4_neighbors(main, pos_x, pos_y):
    count = 0
    if(main[pos_x-1][pos_y] < 255): 
        count+=1
    if(main[pos_x+1][pos_y] < 255): 
        count+=1
    if(main[pos_x][pos_y-1] < 255): 
        count+=1
    if(main[pos_x][pos_y+1] < 255): 
        count+=1
    return count

In [ ]:
def scancluster(image, pos_x, pos_y, cluster):
    image[pos_x][pos_y] = 1
    cluster.append((pos_x, pos_y, find_8_neighbors(image, pos_x, pos_y), find_4_neighbors(image, pos_x, pos_y)))
    
    for i in range(pos_x-1,pos_x+2):
        for j in range(pos_y-1, pos_y+2):
            if image[i][j] == 0:
                scancluster(image, i, j, cluster)

In [ ]:
def process(image, clusters):
    n,m = image.shape
    for i in range(1, n-1):
        for j in range(1, m-1):
            if image[i][j]==0:
                cluster = []
                scancluster(image, i, j, cluster)
                clusters.append(cluster)       

In [ ]:
clusters = []
demo1 = copy.deepcopy(main)
process(demo1, clusters)
print(clusters)

In [ ]:
endpoints = []
for cluster in clusters:
    cluster.sort(key = lambda x: (x[3], x[2]))
    if len(cluster)>1:
#         a = cluster[0][2]
#         b = cluster[0][3]
#         for p in cluster:
#             if p[2]!=a and p[3]!=b:
#                 break
#             endpoints.append((p[0], p[1]))
        for i in range(min(2,len(cluster))):    
            endpoints.append((cluster[i][0], cluster[i][1]))

In [ ]:
X, Y = [], []
for pt in endpoints:
    X.append(pt[1])
    Y.append(pt[0])
X = np.array(X)
Y = np.array(Y)

In [ ]:
len(clusters)

In [ ]:
%matplotlib inline
#plot image and plot dots on end points

implot = plt.imshow(main, 'gray')
plt.scatter(X, Y)
plt.axis('off')
# plt.show()
plt.savefig('EndPoints.jpg')

In [ ]:
plt.imshow(main, 'gray')
row, col = main.shape
#print(main.shape)
total_points = []
for i in range(row):
    for j in range(col):
        if main[i][j] == 0:
            total_points.append((i,j))
total_points.extend(endpoints)
total_points = sorted(total_points, key=lambda x:(x[1],x[0]))

In [ ]:
print(list(total_points))

In [ ]:
plt.imshow(main,'gray')
x, y = [], []
for i in total_points:
    x.append(i[1])
    y.append(i[0])
x = np.array(x)
y = np.array(y)
s = plt.scatter(x, y,s=15,color='black')
# import imageio
plt.axis('off')
plt.savefig('Filled.jpg')